In [1]:
# Análisis HRV y HRT

## Datos en formato DATAFRAME

El primer paso es la lectura de los datos y la conversión en un dataframe de PANDAS

In [2]:
#read data
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


X_data = np.load(r'C:\Users\Margaft\señal Rebeca\PyCardio-master\X_matrix.npy')
#X_data = np.load('X_matrix.npy')

#no borres esta línea de arriba, te he añadido la línea para que puedas llamar desde tu ordenador a los datos
#y vamos comentando y descomentando.

#X_data = np.load(r'C:\Users\Margaft\señal Rebeca\PyCardio-master\X_matrix.npy')

#labels: id, name, Ptot, Pulf, Pvlf, Plf, Phf, lfhf_ratio, scl, ci, cp, to, ts, to_average, ts_average 

labels = ['id', 'name', 'Ptot', 'Pulf', 'Pvlf', 'Plf', 'Phf', 'lfhf_ratio', 'scl', 'ci', 'cp', 'to', 'ts', 'to_average', 'ts_average']
    
#convert into pandas

df = pd.DataFrame(data = X_data,columns = labels)

#change datatypes

df[labels[2:]] = df[labels[2:]].apply(pd.to_numeric)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Margaft\\señal Rebeca\\PyCardio-master\\X_matrix.npy'

Mostramos las 5 primeras filas del dataframe.

In [ ]:
df.head()

Obtenemos un resumen del dataframe. Una de las cosas que podemos apreciar es que tenemos 1095 datos.

In [ ]:
df.describe()

### Estructura de la matriz

Vamos a entender la estructura de la matriz de datos. Cada fila de la matriz corresponde la ocurrencia de un VPC-tachogram dentro de un Holter de 24 horas. Como estamos estudiando los VPC-tachogram aislados, podemos tener más de un VPC-tachogram que pertenezca al mismo paciente.

Las columnas representan las características de cada VPC-tachogram individual:
 * *id* **Identificador de paciente**. Cada paciente en la base de datos tiene un identificador numérico. 
 * *name* **Identificador de paciente - nombre**. Este es un identificador de paciente con iniciales. Lo vamos a eliminar por seguridad.
 * *Ptot* Potencia total de las señal de intervalos RR en los 3 minutos previos. Mide la potencia de la señal de variabilidad.
 * *Pulf* Potencia en la banda de *ultra low frequency*. Esta banda no tiene sentido en 3 minutos.
 * *Pvlf* Potencia en la banda de *very low frequency*.
 * *Plf* Potencia en la banda de *low frequency*.
 * *Phf* Potencia en la banda de *high frequency*.
 * *lfhf_ratio* Ratio de potencias lf/hf
 * *scl* **Sinus cardiac length** corresponde al promedio de los 3 intervalos RR previos al VPC. Mide el ritmo cardiaco al que se encuentra el paciente cuando se produce el VPC.
 * *ci* **Coupling interval** corresponde al intervalo RR del latido VPC.
 * *cp* **Compensatory Pause** corresponde a la pausa compensatoria, esto es, el intervalo RR justo des pués del VPC.
 * *to* **Turbulence Onset** corresponde al parámetro TO, calculado sobre el VPC-tachogram individual en cuestión.
 * *ts* **Turbulence Slope** corresponde al parámetro TS, calculado sobre el VPC-tachogram individual en cuestión.
 * *to_average* **Turbulence Onset** corresponde al parámetro TO, calculado sobre el VPC-tachogram promedio para el paciente. Debe ser igual para todos las filas de la matriz que correspondan a un mismo paciente.
 * *ts_average* **Turbulence Slope** corresponde al parámetro TS, calculado sobre el VPC-tachogram promedio para el paciente. Debe ser igual para todos las filas de la matriz que correspondan a un mismo paciente.

In [ ]:
#remove name col

df.drop(labels = ['name'],axis = 1,inplace = True)

In [ ]:
df.describe()

## Análisis exploratorio de cada una de las variables

En primer lugar vamos a realizar un pequeño análisis de cada una de las variables por separado. De esta forma, tendremos una idea de cuál es la distribución de valores de los diferentes parámetros.

#### Distribución de Ptot

In [ ]:
#Pintamos el histograma

df[df['Ptot']<2000].hist(column = 'Ptot', bins = 50)

#### Distribución de Pvlf


In [ ]:
df.hist(column = 'Pvlf',bins = 50)

## TO DO

Repetir los gráficos para cada una de las variables. Hacer algún comentario sobre cada una de ellas

In [ ]:
df.hist(column = 'Pulf')

Aquí hay que notar que todos los valores son 0. Esto tiene sentido, pues es banda carece de esentido en un segemento de intervalos RR de duración 3 minutos.

#### Potencia en banda LF

In [ ]:
df.hist(column = 'Plf',bins =100)

#### Potencia en banda HF

In [ ]:
df.hist(column = 'Phf')

#### Ratio LF/HF

In [ ]:
df.hist(column = 'lfhf_ratio')

#### SCL antes del VPC

In [ ]:
df.hist(column = 'scl')

#### Copuling Interval del VPC

In [ ]:
df.hist(column = 'ci')

#### Pause Compentasory after VPC

In [ ]:
df.hist(column = 'cp')

#### TS and TO for isolated VPC-tchograms

In [ ]:
df.hist(column = 'to')
df.hist(column = 'ts')

#### Average TS and TO, per patient

In [ ]:
df.hist(column = 'to_average')
df.hist(column = 'ts_average')

### REFLEXIONES SOBRE LOS HISTOGRAMAS

Hay una cosa que me ha llamado mucho la atención al ver los histogramas. Tiene que ver con el hecho de que no se ve casi nada en los histogramas de las potencias. Da la impresión de que tenemos un montón de muestras con un valor muy pequeño, y muy pocas con valores muy grandes. Esto hace que la pinta de los histogramas sea como exponencial. 

Me gustaría comprobar si esto se soluciona aplicando una transformación logarítmica a estos valores. Aquí vamos a tenes problemas con los valores que sean 0. Por eso, descartamos ya la variable PUlf. En las restantes no debería haber valores de 0, salvo error. Vamos a representar lo que ocurre cuando transofrmamos. Esto debería resultar en un histograma más "gaussiano". Esto nos va a permitir luego entender mejor los resultados. 

Realizo yo la transformación para una de las variables, haz el resto. **Ojo: sólo para las de la potencia (excepto ulf)**.

#### Transformación logarítmica de variables de potencia: Plf

In [ ]:
#con este comando se aplica la función np.log a la variable Plf y la guardamos en una nueva variable con el nombre Plf_log

df['Plf_log'] = df['Plf'].apply(np.log)

In [ ]:
df.hist(column='Plf_log',bins = 50)

**Transformación logarítmica de variables de potencia: Ptot**

In [ ]:
df['Ptot_log'] = df['Ptot'].apply(np.log)

In [ ]:
df.hist(column='Ptot_log',bins = 50)

**Transformación logarítmica de variables de potencia: Pvlf**

In [ ]:
df['Pvlf_log'] = df['Pvlf'].apply(np.log)

In [ ]:
df.hist(column='Pvlf_log',bins = 50)

**Transformación logarítmica de variables de potencia: Plf**

In [ ]:
df['Plf_log'] = df['Plf'].apply(np.log)
df.hist(column='Plf_log',bins = 50)

**Transformación logarítmica de variables de potencia: Phf**

In [ ]:
df['Phf_log'] = df['Phf'].apply(np.log)
df.hist(column='Phf_log',bins = 50)

**Transformación logarítmica de variables de potencia: lfhf_ratio**

In [ ]:
df['lfhf_ratio_log'] = df['lfhf_ratio'].apply(np.log)
df.hist(column='lfhf_ratio_log',bins = 50)

## TO DO: realizar las transformaciones de las variables y representar

In [ ]:
#creamos la variable low_risk, que será 1 si el VPC-tachogram pertenece a un pat de low risk
# será 0 si pertenece a uno de high risk y será NaN si no pertenece ni a uno ni a otro
df['low_risk'] = np.NaN

print(np.sum(df['low_risk'].isna()))

df.head()

#asiganmos 1 a los VPC-tachograms de pats que cumplan ts_a >2.5 & to_a < 0

#df.loc[(df['ts_average']>2.5) & (df['to_average']<0),'low_risk'] = 1
df.loc[(df['ts_average']>2.5) & (df['to_average']<0),'low_risk'] = 'Low risk'

#asiganmos 0 a los VPC-tachograms de pats que cumplan ts_a <2.5 & to_a > 0
#df.loc[(df['ts_average']<2.5) & (df['to_average']>0),'low_risk'] = 0
df.loc[(df['ts_average']<2.5) & (df['to_average']>0),'low_risk'] = 'High risk'


print(np.sum(df['low_risk'].isna()))

df.head()

## TO_DO: boxplots de todas las variables (con log y sin log). Menos Pulf (descartada ya) y to y ts.

A partir de ahora $to$ y $ts$ van a ser nuestras variables respuesta, por lo que vamos a intentar relacionar todas las otras variables con estas. 

### Comparación con Boxplots
Vamos a realizar una representación de boxplot. Hecha un vistazo a lo que son los boxplots en la wikipedia [boxplot](https://en.wikipedia.org/wiki/Box_plot). Esto nos va a permitir verificar si existe alguna diferencia entre los valores de HRV, medidos con índices espectrales para los pacientes de alto riesgo vs. los de bajo riesgo. Es una representación muy parecida a la que tu has planteado, pero es más compacta, nos permite ver de un vistazo, y en la misma gráfica, la comparación entre las distribuciones.

In [ ]:
#boxplot for Ptot, and the corresponding log

df.boxplot('Ptot',by='low_risk',figsize = (10,6))
df.boxplot('Ptot_log',by='low_risk',figsize = (10,6))


## Reflexiones sobre los resultados con este primer boxplot

Una cosa que podemos ver es que, cuando no representamos los valores con la transformación logarítmica, la mayoría de los valores los muestra como si fuesen *outliers*. Sin embargo, cuando se realiza la transformación, se puede observar, que al ser la distribución de los valores, "más gaussiana", se pueden analizar las diferencias más sencillamente. 

Yo destacaría que los valores de Ptot, para los pacientes de bajo resgo (1.0, luego podemos cambiar esto en el dataframe, para que aparezcan labels mejor pintados), poseen una Potencia total mayor. Quizá indicando que la HRV es mayor.

Como puedes ver este análisis es más sencillo de realizar cuando utilizamos boxplots.

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de la Ptot correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'Ptot', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Ptot', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Ptot', bins = 50)

#Dibujamos el histograma  de la Ptot_log correspondiente a los tacogramas de bajo riesgo
df2= df[df['low_risk'] == 1]
df2.hist(column = 'Ptot_log', bins = 50)

#Dibujamos el histograma  de la Ptot_log correspondiente a los tacogramas de alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Ptot_log', bins = 50)

#Histograma Ptot_log para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Ptot_log', bins = 50)
"""

### Pvlf

In [ ]:
#boxplot for Pvlf, and the corresponding log

df.boxplot('Pvlf',by='low_risk',figsize = (10,6))
df.boxplot('Pvlf_log',by='low_risk',figsize = (10,6))

**Representación Pvlf y Pvlf_log de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de la Pvlf correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'Pvlf', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Pvlf', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Pvlf', bins = 50)

#Dibujamos el histograma  de la Pvlf_log correspondiente a los tacogramas de bajo riesgo
df2= df[df['low_risk'] == 1]
df2.hist(column = 'Pvlf_log', bins = 50)

#Dibujamos el histograma  de la Pvlf_log correspondiente a los tacogramas de alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Pvlf_log', bins = 50)

#Histograma Pvlf_log para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Pvlf_log', bins = 50)
"""

**Representación Plf y Plf_log de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de la Plf correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'Plf', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Plf', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Plf', bins = 50)

#Dibujamos el histograma  de la Plf_log correspondiente a los tacogramas de bajo riesgo
df2= df[df['low_risk'] == 1]
df2.hist(column = 'Plf_log', bins = 50)

#Dibujamos el histograma  de la Plf_log correspondiente a los tacogramas de alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Plf_log', bins = 50)

#Histograma Plf_log para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Plf_log', bins = 50)
"""

In [ ]:
#boxplot for Plf, and the corresponding log

df.boxplot('Plf',by='low_risk',figsize = (10,6))
df.boxplot('Plf_log',by='low_risk',figsize = (10,6))

**Representación Phf y Phf_log de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de la Phf correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'Phf', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Phf', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Phf', bins = 50)

#Dibujamos el histograma  de la Phf_log correspondiente a los tacogramas de bajo riesgo
df2= df[df['low_risk'] == 1]
df2.hist(column = 'Phf_log', bins = 50)

#Dibujamos el histograma  de la Phf_log correspondiente a los tacogramas de alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'Phf_log', bins = 50)

#Histograma Phf_log para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'Phf_log', bins = 50)
"""

In [ ]:
#boxplot for Phf, and the corresponding log

df.boxplot('Phf',by='low_risk',figsize = (10,6))
df.boxplot('Phf_log',by='low_risk',figsize = (10,6))

**Representación lfhf_ratio y lfhf_ratio_log de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de lfhf_ratio correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'lfhf_ratio', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'lfhf_ratio', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'lfhf_ratio', bins = 50)

#Dibujamos el histograma  de la  lfhf_ratio_log correspondiente a los tacogramas de bajo riesgo
df2= df[df['low_risk'] == 1]
df2.hist(column = 'lfhf_ratio_log', bins = 50)

#Dibujamos el histograma  de la  lfhf_ratio_log correspondiente a los tacogramas de alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'lfhf_ratio_log', bins = 50)

#Histograma  lfhf_ratio_log para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'lfhf_ratio_log', bins = 50)
"""

In [ ]:
#boxplot for Phf, and the corresponding log
df.boxplot('lfhf_ratio',by='low_risk',figsize = (10,6))
df.boxplot('lfhf_ratio_log',by='low_risk',figsize = (10,6))

**Representación scl de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de scl correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'scl', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'scl', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'scl', bins = 50)
"""

df.boxplot('scl',by='low_risk',figsize = (10,6))

**Representación ci de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de  ci correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'ci', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'ci', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'ci', bins = 50)
"""
df.boxplot('ci',by='low_risk',figsize = (10,6))

**Representación cp de los VPC-tachogram de bajo riesgo, alto riesgo y ni uno ni otro**

In [ ]:
"""#Obtenemos un nuevo Dataframe con los tacogramas de bajo riesgo, es decir, low_risk = 1
df2 = df[df['low_risk'] == 1]

#Dibujamos el histograma  de  cp correspondiente a los tacogramas de bajo riesgo
df2.hist(column = 'cp', bins = 50)

#Realizamos lo mismo pero para los correspondientes a alto riesgo
df2= df[df['low_risk'] == 0]
df2.hist(column = 'cp', bins = 50)

#Histograma para los que no pertenecen ni a bajo ni a alto riesgo
df2= df[df['low_risk'].isna()]
df2.hist(column = 'cp', bins = 50)
"""
df.boxplot('cp',by='low_risk',figsize = (10,6))

#  **Relación de to y ts con el resto de variables**

## Reflexiones

En primer lugar, vamos a intentar representar con respecto a $ts$, con este parámetro es con el que suelen salir las cosas con más significado. He representado todos los puntos, y les he puesto color distinto a los que son de bajo riesgo de los que son de alto riesgo, luego los vamos a analizar por separado, porque queda claro que existe una relación, que seguro que podemos medir con una regresión lineal, pero que la recta que vamos a poder dibujar es diferente, dependiendo de si es de bajo riesgo o de alto riesgo. También está claro que si hacemos la representación frente a la variable con log parece que tenemos mejor representación.

## TO DO 
Repetir los scatter plot para todas las demás variables. Cuando tengas esto, le echamos un vistazo y vemos que análisis estadísticos podemos hacer, yo creo que unas cuantas regresiones lineleas separando por low-risk y high-risk, van a salir significativas. Con eso ya estaría toda la parte del análisis :)

Un saludo

**to y Ptot / to y Ptot_log**

Aquí lo que deberíamos utilizar es una representación de [scatter-plot](https://en.wikipedia.org/wiki/Scatter_plot)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Ptot'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Ptot [ms^2]')
plt.ylabel('to')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Ptot_log'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Ptot_log')
plt.ylabel('to')

**to y Plf / to y Plf_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Plf'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Plf [ms^2]')
plt.ylabel('to')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Plf_log'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Plf_log')
plt.ylabel('to')

**to y Pvlf / to y Pvlf_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Pvlf'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Pvlf [ms^2]')
plt.ylabel('to')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Pvlf_log'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Pvlf_log')
plt.ylabel('to')

**to y Phf / to y Phf_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Phf'],y = df['to'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('Phf [ms^2]')
plt.ylabel('to')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Phf_log'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Phf_log')
plt.ylabel('to')

**to y lfhf_ratio / to y lfhf_ratio_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['lfhf_ratio'],y = df['to'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('lfhf_ratio')
plt.ylabel('to')
plt.figure(figsize = (8,6))
plt.scatter(x = df['lfhf_ratio_log'],y = df['to'],c = df['low_risk'] == 'Low risk')
plt.xlabel('lfhf_ratio_log')
plt.ylabel('to')

**to y scl**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['scl'],y = df['to'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('scl [ms]')
plt.ylabel('to')


**to y ci**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['ci'],y = df['to'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('ci [ms]')
plt.ylabel('to')


**to y cp**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['cp'],y = df['to'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('cp [ms]')
plt.ylabel('to')


**ts y Ptot / ts y Ptot_log**

Aquí lo que deberíamos utilizar es una representación de [scatter-plot](https://en.wikipedia.org/wiki/Scatter_plot)

In [ ]:
import matplotlib.pyplot as plt

#df.sort_values(by = ['Ptot'], ascending = True)
#df.plot.scatter(x='Ptot',y='ts',c= (df['low_risk'] == 'Low risk'),colormap = 'spring',figsize = (10,6))
#df.plot.scatter(x='Ptot_log',y='ts',c= (df['low_risk'] == 'Low risk'),colormap = 'spring',figsize = (10,6))

plt.figure(figsize = (8,6))
plt.scatter(x = df['Ptot'],y = df['ts'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('Ptot [ms^2]')
plt.ylabel('ts')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Ptot_log'],y = df['ts'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Ptot_log')
plt.ylabel('ts')

**ts y Pvlf / ts y Pvlf_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Pvlf'],y = df['ts'], c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('Pvlf [ms^2]')
plt.ylabel('ts')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Pvlf_log'],y = df['ts'], c = df['low_risk'] == 'Low risk')
plt.xlabel('Pvlf_log')
plt.ylabel('ts')

**ts y Plf / ts y Plf_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Plf'],y = df['ts'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('Plf [ms^2]')
plt.ylabel('ts')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Plf_log'],y = df['ts'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Plf_log')
plt.ylabel('ts')

**ts y Phf / ts y Phf_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['Phf'],y = df['ts'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('Phf [ms^2]')
plt.ylabel('ts')
plt.figure(figsize = (8,6))
plt.scatter(x = df['Phf_log'],y = df['ts'],c = df['low_risk'] == 'Low risk')
plt.xlabel('Phf_log')
plt.ylabel('ts')

**ts y lfhf_ratio / ts y lfhf_ratio_log**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['lfhf_ratio'],y = df['ts'],c = df['low_risk'] == 'Low risk')
plt.xlabel('lfhf_ratio')
plt.ylabel('ts')
plt.figure(figsize = (8,6))
plt.scatter(x = df['lfhf_ratio_log'],y = df['ts'],c = df['low_risk'] == 'Low risk')
plt.xlabel('lfhf_ratio_log')
plt.ylabel('ts')

**ts y scl**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['scl'],y = df['ts'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('scl [ms]')
plt.ylabel('ts')


**ts y ci**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['ci'],y = df['ts'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('ci [ms]')
plt.ylabel('ts')


**ts y cp**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,6))
plt.scatter(x = df['cp'],y = df['ts'],c = df['low_risk'] == 'Low risk')
#plt.legend()
plt.xlabel('cp [ms]')
plt.ylabel('ts')


## Análisis de regresión lineal

Vamos a realizar el análisis de [regresión lineal](https://es.wikipedia.org/wiki/Regresi%C3%B3n_lineal). También te pediría que le echases un vistazo al capítulo 3 de [ISLR](https://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), sobre todo el apartado 3.1. 

En primer lugar lo que vamos a hacer es separar los dataframes en dos dataframes, uno para low-risk y otro para high-risk, y luego realizar el análisis de regresión linea por separado.

In [ ]:
df_lowrisk = df[df['low_risk'] == 'Low risk']

df_lowrisk.head()

In [ ]:
df_highrisk = df[df['low_risk'] == 'High risk']
df_highrisk.head()

#### Low Risk: Regresión lineal entre Ptot_log y ts

In [ ]:
import numpy as np
import statsmodels.api as sm

#low_risk
X_ptot = df_lowrisk['Ptot_log']
ts = df_lowrisk['ts']

#add intercept
X_ptot = sm.add_constant(X_ptot)

#fit and summary
model = sm.OLS(ts, X_ptot)
results = model.fit()
print(results.summary())

### Interpretación de los resultados


Vamos a realizar una interpretación rápida de los resultados del análisis de regresión:

* 1.- En primer lugar tenemos que verificar si el modelo de regresión lineal en su conjunto es estadísticamente significativo o no. Para ello vamos a donde se indica Prob(F-statistic), si esta probabilidad es menor que 0.05, entonces podemos decir que el modelo es significativo.
* 2.- El R-squared nos dice el porcentaje de la varianza de la variable y (TS) que somos capaces de explicar con la variable Ptot_log. Como vemos no es mucho, apporx un 30%, pero tampoco es bajo.
* 3.- Una vez visto el modelo en su conjunto vamos a ver si cada una de las variables del modelo son significativamente diferentes de cero. En el caso de modelos de regresión simple sólo tenemos una variable, en este caso el coeficiente que acompaña a la variable Ptot_log. También tenemos el intercept, que es la ordenada en el origen. Para ver si son diferente de cero, vamos a analizar el valor de P >|t|, si esta probabilidad es menor que 0.05, entonces podemos decir que el coeficiente que multiplica a la variable es significativamente diferente de 0, y por lo tanto que la variable en cuestión aporta al modelo. Como ocurre en este caso.
* 4.- Luego vamos a ver el valor del coeficiente, que en este caso es 6.26. Lo que nos dice es que cuando aumenta Ptot_log, aumenta TS y además por cada unidad que aumente, aumenta 6.26 TS (ojo Ptot está en unidades logarítmicas).

In [ ]:
#plot
#from matplotlib import rc
#import matplotlib.pyplot as plt


#rc('text', usetex=True)
plt.figure(figsize = (10,8))

plt.scatter(df_lowrisk['Ptot_log'],df_lowrisk['ts'])
plt.xlabel('$\log{(P_{tot})}$',fontsize = 20)
plt.ylabel('$TS$',fontsize = 20)

xx = np.linspace(np.min(df_lowrisk['Ptot_log']),np.max(df_lowrisk['Ptot_log']),100)
xx = sm.add_constant(xx)
yy = results.predict(xx)
plt.plot(xx[:,1],yy,'r--',linewidth = 3)

#text with linear regresion
s = '$TS = {} + $'
plt.text(4, 50, r'$TS = %.2f + %.2f \log({P_{tot}})$' % (results.params[0],results.params[1]), fontsize=20)

#### High Risk: Regresión lineal entre Ptot_log y ts

In [ ]:
import numpy as np
import statsmodels.api as sm

#high_risk
X_ptot = df_highrisk['Ptot_log']
ts = df_highrisk['ts']

#add intercept
X_ptot = sm.add_constant(X_ptot)

#fit and summary
model = sm.OLS(ts, X_ptot)
results = model.fit()
print(results.summary())